In [ ]:
%matplotlib nbagg

In [ ]:
import jefimenko
import numpy as np

In [ ]:
def test_charge_1():
    mapping = jefimenko.Grid(1,   # creat a 1D grid
                delta=[1],  # this is the size of a step in x, y and z in meters
                size=[10],  # this is the size of the grid in meters
                time=3 * 3.36 * 10**-9,  # this is the length of a simulation in secounds
                delta_t=3.36 * 10**-9  # this is the size of a full time step
        )
 
    mapping.Add_Charge([0], Q = 1)
    # jefimenko.plot_grid(mapping)

    jefimenko.simulate(mapping)

    if(mapping.grid['E'][0][1] == [0, 0, 0]).all():
        print("time == 0 R == 1 E field PASS")
    else:
        print("time = 0 R == 1 E field FAIL")

    if(mapping.grid['H'][0][1] == [0, 0, 0]).all():
        print("time == 0 R == 1 H field PASS")
    else:
        print("time == 0 R == 1 H field FAIL")

    test_array = [8987551787.368176, 0, 0]

    if (mapping.grid['E'][1][1] - test_array < .01).all():
        percent = (np.linalg.norm(mapping.grid['E'][1][1] - 8987551787.368176) / 8987551787.368176)
        print("time == 1 R == 1 PASS")
        # print("E field is within " + str(percent) + " percent at time = 1")
        # print("E field is " + str(mapping.grid['E'][1][1]))
    else:
        print("time == 0 R == 1 FAIL")   

    test_array = [2246887946.842, 0, 0]
    if (mapping.grid['E'][1][2] - test_array < .01).all():
        print("time == 1 R == 2 Pass")
    else:
        print("time == 1 R == 2 Fail")
        print(mapping.grid['E'][1][2])
        
    if(mapping.grid['H'][1][1] == [0, 0, 0]).all():
        print("time == 1 R == 1 H field PASS")
    else:
        print("time == 1 R == 1 H field FAIL")
    #for t in range(mapping.time_size):
    #    jefimenko.plot_EM_grid('E', mapping, time=t)

    #field = jefimenko.print_location(mapping.grid['E'][1], [1, 0], mapping)
    #print(field)

In [ ]:
test_charge_1()

In [ ]:
def test_current_1():
    mapping = jefimenko.Grid(1,   # creat a 1D grid
                delta=[1],  # this is the size of a step in x, y and z
                size=[10],  # this is the size of the grid in meters
                time=3 * 3.36 * 10**-9,  # this is the length of a simulation in secounds
                delta_t=3.36 * 10**-9  # this is the size of a full time step
        )
    mapping.Add_Current([0],
                    direction=[0, 0, 1],
                    Amps=1/(np.linalg.norm(mapping.delta)))

    # jefimenko.plot_grid(mapping)

    U_0 = jefimenko.U_0

    for i in range(len(mapping.currents[0])):
        print('amps = ' + str(mapping.currents[0][i].amps))
        print('direction = ' + str(mapping.currents[0][i].direction))
        print('location = '+ str(mapping.currents[0][i].location))
        print('')
    jefimenko.simulate(mapping)

    if(mapping.grid['H'][0][1] == [0, 0, 0]).all():
        print("time == 0 PASS")
    else:
        print("time == 0 Fail")
    test_array = [0, 2*10**-7, 0]

    percent = abs((np.linalg.norm(U_0 * mapping.grid['H'][1][1] - test_array) / (2*10**-7)) * 100)

    if percent <= 1:
        print("time == 1 R == 1 PASS")
    else:
        print("H field is within " + str( percent) + " percent at time = 1")
        print("time = 1 FAIL")
        print(mapping.grid['H'][1][1][1])

    test_array = [0, 1*10**-7, 0]
    percent = abs((np.linalg.norm(U_0 * mapping.grid['H'][2][2] - test_array) / (1*10**-7)) * 100)

    if percent <= 1:
        print("time == 2 R == 2 PASS")
    else:
        print("H field is within " + str( percent) + " percent at time = 2 R == 2")
        print("time = 2 FAIL")
        print(mapping.grid['H'][2][2][1])   


In [ ]:
test_current_1()

In [ ]:
def test_current_2():
    # time = 99
    time = 50
    #r = 10
    r = 50

    mapping = jefimenko.Grid(1,   # creat a 1D grid
                delta=[1],  # this is the size of a step in x, y and z
                size=[r + 1],  # this is the size of the grid in meters
                time=(time + 1) * 3.36 * 10**-9,  # this is the length of a simulation in secounds
                delta_t=3.36 * 10**-9  # this is the size of a full time step
        )
    delta_t = 3.36 * 10**-9
    mapping.Add_Current([0],
                    direction=[0, 1, 0],
                    Amps=1/(np.linalg.norm(mapping.delta)))

    #jefimenko.plot_grid(mapping)

    for i in range(len(mapping.currents[0])):
        print('amps = ' + str(mapping.currents[0][i].amps))
        print('direction = ' + str(mapping.currents[0][i].direction))
        print('location = '+ str(mapping.currents[0][i].location))
        print('')

    w = 2 * np.pi

    C_0 = jefimenko.C_0
    U_0 = jefimenko.U_0
    E_0 = jefimenko.E_0
    
    pass_count = 0
    fail_count = 0
    pass_count_E = 0
    fail_count_E = 0
    
    scale = np.linalg.norm([1]) / (4 * np.pi)
    
    I_0 = 1
    for t in range(time + 1):
        mapping.Modify_Current(0, time=t * delta_t, amps=I_0 * np.sin(t * delta_t * w))

    jefimenko.simulate(mapping)
    # for t in range(mapping.time_size):
    #     jefimenko.plot_EM_grid('H', mapping, time=t)

    # for t in range(mapping.time_size):
    # for t in range(10):
    #    jefimenko.plot_EM_grid('H', mapping, time=t)

    # jefimenko.plot_EM_grid('H', mapping, time=time-1)

    
    for test_time in range(time + 1):
        for test_r in range(1, test_time):

            test_array_E_1 = [0,- I_0 * w * np.cos(w * (test_time - test_r / C_0)) / (4 * np.pi * E_0 * test_r * C_0**2), 0 ]
            test_array_E_2 = [0, -I_0 * np.sin(w * (test_time * delta_t - test_r / C_0)) / (test_r**2 * E_0 * 2 * np.pi * C_0), 0]
            
            test_array_E = [test_array_E_1[0] + test_array_E_2[0], test_array_E_1[1] + test_array_E_2[1], test_array_E_1[2] + test_array_E_2[2]]

            test_array_H_1 = [0, 0, - 1 * I_0 * w * np.cos(w * (test_time * delta_t - test_r / C_0)) / (4 * np.pi * test_r * C_0)]
            test_array_H_2 = [0, 0, - scale * I_0 * np.sin(w * (test_time * delta_t - test_r / C_0)) * 2 / test_r ]

            test_array_H = [test_array_H_1[0] + test_array_H_2[0], test_array_H_1[1] + test_array_H_2[1], test_array_H_1[2] + test_array_H_2[2]]

            percent_E = abs((np.linalg.norm(mapping.grid['E'][test_time][test_r] - test_array_E) / np.linalg.norm(test_array_E)) * 100)
            percent_H = abs((np.linalg.norm(mapping.grid['H'][test_time][test_r] - test_array_H) / np.linalg.norm(test_array_H)) * 100)

            if percent_E <= 5:
                pass_count_E  = pass_count_E + 1

            else:
                fail_count_E = fail_count_E + 1

            if percent_H <= 5:
                pass_count  = pass_count + 1
            else:
                fail_count = fail_count + 1

    print('E field pass count = ' + str(pass_count_E))
    print('E fiels fail count = ' + str(fail_count_E))
    if fail_count_E < 120:
        print('this is curently exseptable')
    print() 
    print('H field pass count = ' + str(pass_count))
    print('H field fail count = ' + str(fail_count))
    if fail_count < 120:
        print('this is curently exseptable')
    

In [ ]:
test_current_2()


In [ ]:
def test_current_3():
    time = 50
    r = 50

    mapping = jefimenko.Grid(1,   # creat a 1D grid
                delta=[1],  # this is the size of a step in x, y and z
                size=[r + 1],  # this is the size of the grid in meters
                time=(time + 1) * 3.36 * 10**-9,  # this is the length of a simulation in secounds
                delta_t=3.36 * 10**-9  # this is the size of a full time step
        )
    delta_t = 3.36 * 10**-9
    mapping.Add_Current([0],
                    direction=[0, 0, 1],
                    Amps=1/(np.linalg.norm(mapping.delta)))

    for i in range(len(mapping.currents[0])):
        print('amps = ' + str(mapping.currents[0][i].amps))
        print('direction = ' + str(mapping.currents[0][i].direction))
        print('location = '+ str(mapping.currents[0][i].location))
        print('')

    w = 2 * np.pi

    C_0 = jefimenko.C_0
    U_0 = jefimenko.U_0
    E_0 = jefimenko.E_0
    
    pass_count = 0
    fail_count = 0
    pass_count_E = 0
    fail_count_E = 0
    
    scale = np.linalg.norm([1]) / (4 * np.pi)
    
    I_0 = 1
    for t in range(time + 1):
        mapping.Modify_Current(0, time=t * delta_t, amps=I_0 * np.sin(t * delta_t * w))

    jefimenko.simulate(mapping)
    
    for test_time in range(time + 1):
        for test_r in range(1, test_time):

            test_array_E_1 = [0, 0,- I_0 * w * np.cos(w * (test_time - test_r / C_0)) / (4 * np.pi * E_0 * test_r * C_0**2)]
            test_array_E_2 = [0, 0, -I_0 * np.sin(w * (test_time * delta_t - test_r / C_0)) / (test_r**2 * E_0 * 2 * np.pi * C_0)]

            test_array_E = [test_array_E_1[0] + test_array_E_2[0], test_array_E_1[1] + test_array_E_2[1], test_array_E_1[2] + test_array_E_2[2]]

            test_array_H_1 = [0, 1 * I_0 * w * np.cos(w * (test_time * delta_t - test_r / C_0)) / (4 * np.pi * test_r * C_0), 0]
            test_array_H_2 = [0, scale * I_0 * np.sin(w * (test_time * delta_t - test_r / C_0)) * 2 / test_r, 0]

            test_array_H = [test_array_H_1[0] + test_array_H_2[0], test_array_H_1[1] + test_array_H_2[1], test_array_H_1[2] + test_array_H_2[2]]

            percent_E = abs((np.linalg.norm(mapping.grid['E'][test_time][test_r] - test_array_E) / np.linalg.norm(test_array_E)) * 100)

            percent_H = abs((np.linalg.norm(mapping.grid['H'][test_time][test_r] - test_array_H) / np.linalg.norm(test_array_H)) * 100)

            if percent_E <= 5:
                pass_count_E  = pass_count_E + 1

            else:
                fail_count_E = fail_count_E + 1


            if percent_H <= 5:
                pass_count  = pass_count + 1
            else:
                fail_count = fail_count + 1

    print('E field pass count = ' + str(pass_count_E))
    print('E fiels fail count = ' + str(fail_count_E))
    if fail_count_E < 120:
        print('this is curently exseptable')

    print()

    print('H field pass count = ' + str(pass_count))
    print('H field fail count = ' + str(fail_count))
    if fail_count < 120:
        print('this is curently exseptable')
    else:
        print('there is room for improvment')

In [ ]:
test_current_3()